#### Arnoldi iteration

Previously we see that Arnoldi iteration can product

$$AQ_n=Q_{n+1}H_{n+1, n}$$

and

$$H_n = Q_n^TAQ_n$$

where the eigenvalues of $H_n$, which is in upper Hessenberg form, approximate eigenvalues of $A$

#### Lanczos iteration for symmetric matrices

Lanczos iteration is Arnoldi iteration in special case that $A$ is `symmetric`, we can see that

$$H_n=Q_n^TAQ_n$$

is also symmetric

For $H_n$ that is both symmetric and upper Hessenberg, it must be `tridiagonal` and we denote it as $T_n$

$AQ_n=Q_{n+1}H_{n+1,n}$ becomes

$$AQ_n = A\begin{bmatrix}q_1 & q_2 & \cdots q_n\end{bmatrix}=Q_{n+1}T_{n+1,n}=\begin{bmatrix}q_1 & q_2 & \cdots q_{n+1}\end{bmatrix}\begin{bmatrix}t_{11} & t_{12} & & & \\
t_{21} & t_{22} & t_{23} & & \\ & t_{32} & t_{33} & \ddots & \\
 & & \ddots & \ddots & t_{n-1, n} \\ & & & t_{n, n-1} & t_{nn} \\ & & & & t_{n+1,n}\end{bmatrix}$$

#### Example

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [2]:
def arnoldi_iteration(A, b, n, tol=1e-10):
    m = A.shape[0]
    Q = np.zeros((m, n+1))
    H = np.zeros((n+1, n))
    Q[:, 0] = b / np.linalg.norm(b)

    for i in range(n):
        v = A @ Q[:, i]

        for j in range(i+1):
            H[j, i] = Q[:, j] @ v
            v -= H[j, i] * Q[:, j]

        H[i+1, i] = np.linalg.norm(v)
        if H[i+1, i] < tol:
            break
        Q[:, i+1] = v / H[i+1, i]

    return Q, H

In [3]:
def lanczos_iteration(A, b, n, tol=1e-10):
    m = A.shape[0]
    Q = np.zeros((m, n+1))
    T = np.zeros((n+1, n))
    Q[:, 0] = b / np.linalg.norm(b)

    for i in range(n):
        v = A @ Q[:, i]

        for j in range(max(i-1, 0), i+1):
            T[j, i] = Q[:, j] @ v
            v -= T[j, i] * Q[:, j]

        T[i+1, i] = np.linalg.norm(v)
        if T[i+1, i] < tol:
            break
        Q[:, i+1] = v / T[i+1, i]

    return Q, T

In [4]:
np.random.seed(42)

m = 10
A = np.random.rand(m, m)

# Turn into symmetric matrix
A = A @ A.T
b = np.random.rand(m)

start_time = time.time()
Q_arnoldi, H = arnoldi_iteration(A, b, 8)
print(f'Time taken Arnoldi: {time.time() - start_time}')

start_time = time.time()
Q_lanczos, T = lanczos_iteration(A, b, 8)
print(f'Time taken Lanczos: {time.time() - start_time}')

Time taken Arnoldi: 0.0009453296661376953
Time taken Lanczos: 0.0006623268127441406


In [5]:
print(f'Q {Q_arnoldi.shape}\n', Q_arnoldi)
print('\nCheck orthogonality in Q:\n', Q_arnoldi.T @ Q_arnoldi)
print(f'\nH {H.shape}\n', H)

print('\nAQ:\n', A @ Q_arnoldi[:,:-1])
print('\nQH:\n', Q_arnoldi @ H)
print('\nDifference:\n', np.linalg.norm(A @ Q_arnoldi[:,:-1] - Q_arnoldi @ H))

Q (10, 9)
 [[ 0.0201  0.5251  0.1640  0.5147  0.1086 -0.1991  0.1841  0.0538  0.5337]
 [ 0.4069 -0.1437  0.6166 -0.0062  0.3281 -0.4549 -0.1552 -0.2341 -0.1650]
 [ 0.2010  0.1947 -0.1932 -0.0561  0.1163  0.1187  0.2863 -0.3117  0.2269]
 [ 0.3251  0.1705 -0.4695 -0.0597  0.1367 -0.4753  0.3703  0.3225 -0.3841]
 [ 0.5802 -0.2966 -0.0893  0.4757 -0.3021  0.1777 -0.2471  0.3526  0.0883]
 [ 0.1594  0.4541  0.1088 -0.5631 -0.2629 -0.1263 -0.4121  0.3490  0.2226]
 [ 0.2624  0.1285  0.4689 -0.1662 -0.2961  0.4018  0.5961  0.0905 -0.1878]
 [ 0.4830 -0.1101 -0.2951 -0.2935  0.0467  0.1136 -0.0282 -0.4692  0.3336]
 [ 0.1463  0.3483 -0.0300  0.0631  0.6242  0.5381 -0.2738  0.1314 -0.2760]
 [ 0.0492  0.4387 -0.0866  0.2593 -0.4566 -0.0443 -0.2469 -0.4995 -0.4570]]

Check orthogonality in Q:
 [[ 1.0000  0.0000  0.0000 -0.0000  0.0000 -0.0000  0.0000 -0.0000  0.0000]
 [ 0.0000  1.0000  0.0000 -0.0000  0.0000 -0.0000  0.0000 -0.0000  0.0000]
 [ 0.0000  0.0000  1.0000  0.0000 -0.0000  0.0000 -0.0000  0

In [6]:
print(f'Q {Q_lanczos.shape}\n', Q_lanczos)
print('\nCheck orthogonality in Q:\n', Q_lanczos.T @ Q_lanczos)
print(f'\nT {T.shape}\n', T)

print('\nAQ:\n', A @ Q_lanczos[:,:-1])
print('\nQT:\n', Q_lanczos @ T)
print('\nDifference:\n', np.linalg.norm(A @ Q_lanczos[:,:-1] - Q_lanczos @ T))

Q (10, 9)
 [[ 0.0201  0.5251  0.1640  0.5147  0.1086 -0.1991  0.1841  0.0538  0.5337]
 [ 0.4069 -0.1437  0.6166 -0.0062  0.3281 -0.4549 -0.1552 -0.2341 -0.1650]
 [ 0.2010  0.1947 -0.1932 -0.0561  0.1163  0.1187  0.2863 -0.3117  0.2269]
 [ 0.3251  0.1705 -0.4695 -0.0597  0.1367 -0.4753  0.3703  0.3225 -0.3841]
 [ 0.5802 -0.2966 -0.0893  0.4757 -0.3021  0.1777 -0.2471  0.3526  0.0883]
 [ 0.1594  0.4541  0.1088 -0.5631 -0.2629 -0.1263 -0.4121  0.3490  0.2226]
 [ 0.2624  0.1285  0.4689 -0.1662 -0.2961  0.4018  0.5961  0.0905 -0.1878]
 [ 0.4830 -0.1101 -0.2951 -0.2935  0.0467  0.1136 -0.0282 -0.4692  0.3336]
 [ 0.1463  0.3483 -0.0300  0.0631  0.6242  0.5381 -0.2738  0.1314 -0.2760]
 [ 0.0492  0.4387 -0.0866  0.2593 -0.4566 -0.0443 -0.2469 -0.4995 -0.4570]]

Check orthogonality in Q:
 [[ 1.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000]
 [ 0.0000  1.0000  0.0000 -0.0000  0.0000  0.0000  0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  1.0000  0.0000 -0.0000  0.0000  0.0000  0

In [7]:
print('Q Arnoldi vs Lanczos', np.linalg.norm(Q_arnoldi - Q_lanczos))
print('H Arnoldi vs T Lanczos', np.linalg.norm(H - T))

Q Arnoldi vs Lanczos 1.0207107506613675e-05
H Arnoldi vs T Lanczos 1.1905222348127529e-11


#### Reorthogonalization

We see Lanczos iteration suffers from loss of orthogonality due to the fact that it only orthogonalizes new vector against last two vectors in $Q$

We can manually do reorthogonalization to fix this problem

In [8]:
def lanczos_iteration_reortho(A, b, n, tol=1e-10):
    m = A.shape[0]
    Q = np.zeros((m, n+1))
    T = np.zeros((n+1, n))
    Q[:, 0] = b / np.linalg.norm(b)

    for i in range(n):
        v = A @ Q[:, i]

        for j in range(max(i-1, 0), i+1):
            T[j, i] = Q[:, j] @ v
            v -= T[j, i] * Q[:, j]

        # Manually reorthogonalize against all previous vectors
        for j in range(i+1):
            correction = Q[:, j] @ v
            v -= correction * Q[:, j]
            T[j, i] += correction

        T[i+1, i] = np.linalg.norm(v)
        if T[i+1, i] < tol:
            break
        Q[:, i+1] = v / T[i+1, i]

    return Q, T

In [9]:
Q_lanczos, T = lanczos_iteration_reortho(A, b, 8)

print('Q Arnoldi vs Lanczos', np.linalg.norm(Q_arnoldi - Q_lanczos))
print('H Arnoldi vs T Lanczos', np.linalg.norm(H - T))

Q Arnoldi vs Lanczos 3.1000101053528675e-14
H Arnoldi vs T Lanczos 4.0623207027424685e-13


Double check for other results

In [10]:
print(f'Q {Q_lanczos.shape}\n', Q_lanczos)
print('\nCheck orthogonality in Q:\n', Q_lanczos.T @ Q_lanczos)
print(f'\nT {T.shape}\n', T)

print('\nAQ:\n', A @ Q_lanczos[:,:-1])
print('\nQT:\n', Q_lanczos @ T)
print('\nDifference:\n', np.linalg.norm(A @ Q_lanczos[:,:-1] - Q_lanczos @ T))

Q (10, 9)
 [[ 0.0201  0.5251  0.1640  0.5147  0.1086 -0.1991  0.1841  0.0538  0.5337]
 [ 0.4069 -0.1437  0.6166 -0.0062  0.3281 -0.4549 -0.1552 -0.2341 -0.1650]
 [ 0.2010  0.1947 -0.1932 -0.0561  0.1163  0.1187  0.2863 -0.3117  0.2269]
 [ 0.3251  0.1705 -0.4695 -0.0597  0.1367 -0.4753  0.3703  0.3225 -0.3841]
 [ 0.5802 -0.2966 -0.0893  0.4757 -0.3021  0.1777 -0.2471  0.3526  0.0883]
 [ 0.1594  0.4541  0.1088 -0.5631 -0.2629 -0.1263 -0.4121  0.3490  0.2226]
 [ 0.2624  0.1285  0.4689 -0.1662 -0.2961  0.4018  0.5961  0.0905 -0.1878]
 [ 0.4830 -0.1101 -0.2951 -0.2935  0.0467  0.1136 -0.0282 -0.4692  0.3336]
 [ 0.1463  0.3483 -0.0300  0.0631  0.6242  0.5381 -0.2738  0.1314 -0.2760]
 [ 0.0492  0.4387 -0.0866  0.2593 -0.4566 -0.0443 -0.2469 -0.4995 -0.4570]]

Check orthogonality in Q:
 [[ 1.0000 -0.0000 -0.0000  0.0000 -0.0000  0.0000  0.0000  0.0000 -0.0000]
 [-0.0000  1.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000]
 [-0.0000 -0.0000  1.0000 -0.0000 -0.0000  0.0000  0.0000  0